In [1]:
import rasterio
from pathlib import Path
from tqdm import tqdm
import shutil

In [10]:
def bounds_contains(outer, inner, atol=0.0):
    L0, B0, R0, T0 = outer
    L1, B1, R1, T1 = inner
    return (L0 <= L1 + atol) and (B0 <= B1 + atol) and \
           (R0 + atol >= R1) and (T0 + atol >= T1)

In [13]:
images_folder = Path('data/TreeAI_Swiss/images')
uav_images = list(images_folder.rglob("*.tif"))
images_num = len(uav_images)

sateEmbed = Path('data/TreeAI_Swiss/Satellite_Embedding')
sateEmbed_images = list(sateEmbed.rglob("*.tif"))
sateEmbed_num = len(sateEmbed_images)

assert sateEmbed_num == images_num, "The number of Satellite Embedding data shoud be the same as of UAV images."

In [17]:
for sate_embed in sateEmbed_images:
    with rasterio.open(sate_embed) as src:
        sate_metadata = src.meta
        sate_bounds = src.bounds

    uav_image_path = Path(str(sate_embed).
                          replace("Satellite_Embedding","images"))
    with rasterio.open(uav_image_path) as f:
        uav_metadata = f.meta
        uav_bounds = f.bounds

    assert uav_metadata['crs'] == sate_metadata['crs'], "The CRS of UAV images should be the same as the one of Satellite Embedding data"

    contain = bounds_contains(outer=sate_bounds, inner=uav_bounds)
    if not contain:
        print(f"For {str(sate_embed)}, not completely contain uav image.")
